In [3]:
import pandas as pd
from utils import transformers_bert_completions, load_splits, load_models
from utils_model_sampling import sample_across_models, hyperparameter_utils

from yyy_analysis import examples_figure
import numpy as np

import os
from os.path import join, exists

import childespy
%load_ext autotime
%load_ext line_profiler
%pdb

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

Automatic pdb calling has been turned ON
time: 3.85 ms (started: 2022-01-28 21:36:46 -08:00)


In [4]:
import configuration
config = configuration.Config()

time: 2.25 ms (started: 2022-01-28 21:36:46 -08:00)


In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"]="" #hangs as of 1/26 if GPU is enabled on workstation

# Success Example

In [5]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



time: 1.9 s (started: 2022-01-28 21:36:46 -08:00)


## Load phonology and context

In [ ]:
all_tokens_phono = load_splits.load_phono()

time: 9.18 s (started: 2022-01-28 21:36:48 -08:00)


In [ ]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


time: 42.9 ms (started: 2022-01-28 21:36:57 -08:00)


In [ ]:
target_transcript_id = 42336 # Corresponds to the success_idx 

time: 372 µs (started: 2022-01-28 21:36:57 -08:00)


In [ ]:
# Broader context of sentences
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


time: 38 ms (started: 2022-01-28 21:36:57 -08:00)


## Estimate Prior and Posterior Probability

In [ ]:
# Changed this notebook to use the +/- 20 context newly generated versions.

# Written to match load_models.query_model_title
default_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : False,
    'context_num' : 20,
}

childes_all_title = load_models.query_model_title(model_type = 'childes', **default_args)
adult_all_title = load_models.query_model_title(model_type = 'adult', **default_args)
unigram_title = 'CHILDES Unigram'
flat_title = 'Flat Unigram'

time: 1.23 ms (started: 2022-01-28 21:36:57 -08:00)


In [ ]:
from utils import wfst
from utils_model_sampling import hyperparameter_utils

time: 7.19 ms (started: 2022-01-28 21:36:58 -08:00)


In [104]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)
imp.reload(wfst)
imp.reload(hyperparameter_utils)

/home/stephan/notebooks/nicole/child-directed-listening/yyy_analysis/examples_figure.py:8: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/sample_across_models.py:8: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py:14: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/hyperparameter_utils.py:9: ResourceWarning: unclosed 

<module 'utils_model_sampling.hyperparameter_utils' from '/home/stephan/notebooks/nicole/child-directed-listening/utils_model_sampling/hyperparameter_utils.py'>

time: 23.4 ms (started: 2022-01-29 09:37:37 -08:00)


In [105]:
# CDL + Context +/- 20 is needed
# BERT + Context +/- 20 is needed
# Childes on train data.

# How to load properly with sample across models?
which_models = [
    # ('all', 'all', False, 0, 'data_unigram'),
    ('all', 'all', False, 20, 'childes'),
    # ('all', 'all', False, 20, 'adult'),
    #('all', 'all', False, 0, 'flat_unigram'),
]

time: 747 µs (started: 2022-01-29 09:37:37 -08:00)


In [ ]:
raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, which_models, True)

Running model CHILDES BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...


/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ats_end[0,2] = ''
/home/stephan/notebooks/nicole/child-directed-listening/utils/wfst.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Processing lambda value 1 of 20
> /home/stephan/notebooks/nicole/child-directed-listening/utils/transformers_bert_completions.py(736)get_posteriors()
    734     import pdb
    735     pdb.set_trace()
--> 736     return(prior_data)
    737 
    738 



ipdb>  prior_data['scores']['posterior_probabilities']


*** KeyError: 'posterior_probabilities'


ipdb>  prior_data['scores']['posterior_probability']


997718    0.999827
997719    0.880881
997720    0.893254
997721    0.984112
Name: posterior_probability, dtype: float64


ipdb>  quit()


BdbQuit: 

> /usr/lib/python3.6/bdb.py(70)dispatch_line()
     68         if self.stop_here(frame) or self.break_here(frame):
     69             self.user_line(frame)
---> 70             if self.quitting: raise BdbQuit
     71         return self.trace_dispatch
     72 



In [ ]:
scores_across_models = pd.concat(raw_scores_across_models)

In [ ]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'wfst']

In [22]:
np.unique(scores_across_models.likelihood_type)

array(['wfst'], dtype=object)

time: 4.38 ms (started: 2022-01-28 22:03:19 -08:00)


In [30]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','posterior_probability','token','likelihood_type','prior_rank','posterior_rank']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,posterior_probability,token,likelihood_type,prior_rank,posterior_rank
997721,"CHILDES BERT without tags, , +-20 utts context",see go play look read know watch write mommy do,0.4728042048364949 0.09090639960534466 0.03111...,see go play look read know watch write mommy do,0.4728042 0.0909064 0.031111874 0.022768175 0....,0.018506,0.018506,read,wfst,4,4.0


time: 32.6 ms (started: 2022-01-29 07:48:59 -08:00)


In [24]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
print('CDL+Context Prior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Prior


'see (0.47) go (0.09) play (0.03) look (0.02) read (0.02) know (0.02) watch (0.02) write (0.01) mommy (0.01) do (0.01)'

time: 14.9 ms (started: 2022-01-28 22:03:24 -08:00)


In [25]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('CDL+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Posterior


'see (0.47) go (0.09) play (0.03) look (0.02) read (0.02) know (0.02) watch (0.02) write (0.01) mommy (0.01) do (0.01)'

time: 18.3 ms (started: 2022-01-28 22:03:26 -08:00)


In [26]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

print('BERT+Context Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

BERT+Context Prior


'read (0.49) see (0.28) play (0.04) know (0.04) look (0.02) go (0.01) watch (0.01) help (0.01) try (0.01) talk (0.01)'

time: 20.1 ms (started: 2022-01-28 22:03:49 -08:00)


In [27]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('BERT+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],4))+')' for i in range(len(words))]) 

BERT+Context Posterior


'read (0.4879) see (0.2761) play (0.0368) know (0.0363) look (0.0153) go (0.012) watch (0.01) help (0.0083) try (0.008) talk (0.0072)'

time: 17.1 ms (started: 2022-01-28 22:03:53 -08:00)


In [28]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token', 'prior_rank', 'posterior_rank']]
#print(success_example)
print('CHILDES-1Gram Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1Gram Prior


'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 22 ms (started: 2022-01-28 22:03:55 -08:00)


In [29]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('CHILDES-1gram Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1gram Posterior


'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 17.2 ms (started: 2022-01-28 22:03:58 -08:00)


In [ ]:
success_example = scores_across_models.loc[(scores_across_models.model == flat_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example
print('UniformPrior Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('UniformPrior Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

# Failure Example

In [ ]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)

In [ ]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, which_models, False)
scores_across_models = pd.concat(raw_scores_across_models)

In [ ]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','posterior_rank']]
yyy_example

In [ ]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

In [ ]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]